## 1. Setup and Configuration

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# SentenceTransformers and evaluation imports
from sentence_transformers import SentenceTransformer, evaluation
import torch

# Sklearn metrics
from sklearn.metrics import (
    roc_auc_score, 
    average_precision_score,
    precision_recall_curve,
    roc_curve,
    f1_score,
    precision_score,
    recall_score
)
from scipy.stats import spearmanr, pearsonr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Imports complete")

In [ ]:
# Auto-detect GPU and configure batch size
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

if device == 'cuda':
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3  # GB
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {gpu_memory:.2f} GB")
    
    # Auto-configure batch size based on available memory
    if gpu_memory >= 24:
        BATCH_SIZE = 128
    elif gpu_memory >= 16:
        BATCH_SIZE = 96
    elif gpu_memory >= 12:
        BATCH_SIZE = 64
    elif gpu_memory >= 8:
        BATCH_SIZE = 48
    elif gpu_memory >= 6:
        BATCH_SIZE = 32
    else:
        BATCH_SIZE = 16
    
    print(f"Auto-configured batch size: {BATCH_SIZE}")
else:
    BATCH_SIZE = 16  # Conservative for CPU
    print(f"CPU mode - using conservative batch size: {BATCH_SIZE}")

# Paths
DATA_DIR = Path('data_new')
MODELS_DIR = Path('models')
RESULTS_DIR = MODELS_DIR / 'results'
RESULTS_DIR.mkdir(exist_ok=True)

print(f"\n✓ Configuration complete")

## 2. Load and Prepare Data

In [ ]:
# Load ServiceNow incident data
data_path = DATA_DIR / 'SNow_incident_ticket_data.csv'
df = pd.read_csv(data_path)

print(f"Loaded {len(df)} ServiceNow incidents")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Combine short_description and description for complete context
def create_combined_text(row):
    """Combine short_description and description with proper handling of NaN"""
    short_desc = str(row.get('short_description', '')).strip() if pd.notna(row.get('short_description')) else ''
    desc = str(row.get('description', '')).strip() if pd.notna(row.get('description')) else ''
    
    if short_desc and desc:
        return f"{short_desc}. {desc}"
    elif short_desc:
        return short_desc
    elif desc:
        return desc
    else:
        return ''

df['combined_text'] = df.apply(create_combined_text, axis=1)

# Filter out empty texts
df = df[df['combined_text'].str.len() > 10].reset_index(drop=True)

print(f"After filtering: {len(df)} valid incidents")
print(f"\nSample combined text:")
print(df['combined_text'].iloc[0])

## 3. Generate Test Pairs

Creates three types of test pairs:
1. **Positive pairs**: Same category/similar incidents (label=1)
2. **Easy negatives**: Different categories (label=0)
3. **Hard negatives**: Different categories but high TF-IDF similarity (label=0)

This adversarial test set ensures robust evaluation.

In [ ]:
def generate_test_pairs(df: pd.DataFrame, 
                       num_positives: int = 500,
                       num_easy_negatives: int = 500,
                       num_hard_negatives: int = 500,
                       random_state: int = 42) -> Tuple[List[str], List[str], List[int]]:
    """
    Generate test pairs for evaluation.
    
    Returns:
        texts1, texts2, labels
    """
    np.random.seed(random_state)
    
    texts1, texts2, labels = [], [], []
    
    # Check if we have category information
    has_categories = 'category' in df.columns
    
    if has_categories:
        print("Using category-based pair generation")
        categories = df['category'].dropna().unique()
        print(f"Found {len(categories)} categories")
        
        # 1. Positive pairs - same category
        for _ in range(num_positives):
            cat = np.random.choice(categories)
            cat_incidents = df[df['category'] == cat]
            if len(cat_incidents) >= 2:
                idx1, idx2 = np.random.choice(cat_incidents.index, size=2, replace=False)
                texts1.append(df.loc[idx1, 'combined_text'])
                texts2.append(df.loc[idx2, 'combined_text'])
                labels.append(1)
        
        # 2. Easy negatives - different categories
        for _ in range(num_easy_negatives):
            cat1, cat2 = np.random.choice(categories, size=2, replace=False)
            incidents1 = df[df['category'] == cat1]
            incidents2 = df[df['category'] == cat2]
            if len(incidents1) > 0 and len(incidents2) > 0:
                idx1 = np.random.choice(incidents1.index)
                idx2 = np.random.choice(incidents2.index)
                texts1.append(df.loc[idx1, 'combined_text'])
                texts2.append(df.loc[idx2, 'combined_text'])
                labels.append(0)
    else:
        print("No category column found - using random pair generation")
        # Random positive pairs (assume some similarity in random selection)
        for _ in range(num_positives):
            idx1, idx2 = np.random.choice(len(df), size=2, replace=False)
            texts1.append(df.loc[idx1, 'combined_text'])
            texts2.append(df.loc[idx2, 'combined_text'])
            labels.append(1)
        
        # Random negative pairs
        for _ in range(num_easy_negatives):
            idx1, idx2 = np.random.choice(len(df), size=2, replace=False)
            texts1.append(df.loc[idx1, 'combined_text'])
            texts2.append(df.loc[idx2, 'combined_text'])
            labels.append(0)
    
    # 3. Hard negatives - high TF-IDF similarity but different incidents
    print("\nGenerating hard negatives with TF-IDF...")
    sample_size = min(2000, len(df))  # Limit for performance
    sample_df = df.sample(n=sample_size, random_state=random_state)
    
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(sample_df['combined_text'])
    
    hard_neg_count = 0
    attempts = 0
    max_attempts = num_hard_negatives * 10
    
    while hard_neg_count < num_hard_negatives and attempts < max_attempts:
        idx1 = np.random.randint(0, len(sample_df))
        
        # Compute similarity with all other incidents
        similarities = sklearn_cosine_similarity(tfidf_matrix[idx1:idx1+1], tfidf_matrix).flatten()
        
        # Find incidents with similarity in range [0.3, 0.6] (hard negatives)
        hard_neg_candidates = np.where((similarities > 0.3) & (similarities < 0.6))[0]
        hard_neg_candidates = hard_neg_candidates[hard_neg_candidates != idx1]
        
        if len(hard_neg_candidates) > 0:
            idx2 = np.random.choice(hard_neg_candidates)
            
            # If we have categories, ensure they're different
            if has_categories:
                cat1 = sample_df.iloc[idx1].get('category')
                cat2 = sample_df.iloc[idx2].get('category')
                if cat1 == cat2:
                    attempts += 1
                    continue
            
            texts1.append(sample_df.iloc[idx1]['combined_text'])
            texts2.append(sample_df.iloc[idx2]['combined_text'])
            labels.append(0)
            hard_neg_count += 1
        
        attempts += 1
    
    print(f"\nGenerated {hard_neg_count} hard negatives ({attempts} attempts)")
    print(f"\nTotal test pairs: {len(labels)}")
    print(f"  Positive pairs: {sum(labels)}")
    print(f"  Negative pairs: {len(labels) - sum(labels)}")
    
    return texts1, texts2, labels

In [ ]:
# Generate test pairs
print("Generating test pairs...\n")
test_texts1, test_texts2, test_labels = generate_test_pairs(
    df,
    num_positives=500,
    num_easy_negatives=500,
    num_hard_negatives=500,
    random_state=42
)

print(f"\n✓ Test set ready: {len(test_labels)} pairs")

## 4. Discover Available Models

In [ ]:
def list_available_models(models_dir: Path) -> Dict[str, List[Path]]:
    """
    List all available trained models in the models directory.
    
    Returns:
        Dictionary mapping model names to their paths
    """
    available_models = {}
    
    # Look for model directories (those containing config.json)
    for model_path in models_dir.iterdir():
        if model_path.is_dir():
            # Check if it's a valid model directory
            config_file = model_path / 'config.json'
            if config_file.exists():
                available_models[model_path.name] = model_path
            else:
                # Check for timestamped subdirectories (e.g., model_20251130_1451)
                for subdir in model_path.iterdir():
                    if subdir.is_dir() and (subdir / 'config.json').exists():
                        model_name = f"{model_path.name}/{subdir.name}"
                        available_models[model_name] = subdir
    
    return available_models

# Discover models
available_models = list_available_models(MODELS_DIR)

print(f"Found {len(available_models)} trained models:\n")
for i, (name, path) in enumerate(sorted(available_models.items()), 1):
    print(f"{i:2d}. {name}")
    print(f"    Path: {path}")

## 5. Model Evaluation Class

In [ ]:
class ModelEvaluator:
    """
    Comprehensive model evaluator for ITSM similarity models.
    """
    
    def __init__(self, batch_size: int = 32):
        self.batch_size = batch_size
        self.results = {}
    
    def evaluate_model(self,
                      model: SentenceTransformer,
                      model_name: str,
                      texts1: List[str],
                      texts2: List[str],
                      labels: List[int],
                      verbose: bool = True) -> Dict:
        """
        Evaluate a model on the given test pairs.
        
        Returns:
            Dictionary with all evaluation metrics
        """
        if verbose:
            print(f"\nEvaluating: {model_name}")
            print("=" * 60)
        
        # Encode pairs
        if verbose:
            print("Encoding text pairs...")
        
        embeddings1 = model.encode(
            texts1,
            batch_size=self.batch_size,
            show_progress_bar=verbose,
            convert_to_numpy=True
        )
        
        embeddings2 = model.encode(
            texts2,
            batch_size=self.batch_size,
            show_progress_bar=verbose,
            convert_to_numpy=True
        )
        
        # Compute normalized cosine similarity
        cosine_scores = np.sum(embeddings1 * embeddings2, axis=1) / (
            np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        )
        
        # Convert labels to numpy array
        labels_array = np.array(labels)
        
        # Compute correlation metrics
        eval_pearson, _ = pearsonr(labels_array, cosine_scores)
        eval_spearman, _ = spearmanr(labels_array, cosine_scores)
        
        # Compute classification metrics
        try:
            roc_auc = roc_auc_score(labels_array, cosine_scores)
            pr_auc = average_precision_score(labels_array, cosine_scores)
        except ValueError as e:
            if verbose:
                print(f"Warning: Could not compute ROC/PR curves: {e}")
            roc_auc = pr_auc = None
        
        # Find optimal threshold for F1 score
        thresholds = np.linspace(0, 1, 100)
        f1_scores = []
        
        for threshold in thresholds:
            predictions = (cosine_scores >= threshold).astype(int)
            if len(np.unique(predictions)) > 1:  # Avoid division by zero
                f1 = f1_score(labels_array, predictions)
                f1_scores.append(f1)
            else:
                f1_scores.append(0)
        
        best_threshold_idx = np.argmax(f1_scores)
        best_threshold = thresholds[best_threshold_idx]
        best_f1 = f1_scores[best_threshold_idx]
        
        # Compute metrics at best threshold
        predictions = (cosine_scores >= best_threshold).astype(int)
        precision = precision_score(labels_array, predictions)
        recall = recall_score(labels_array, predictions)
        
        # Store results
        results = {
            'model_name': model_name,
            'spearman': eval_spearman,
            'pearson': eval_pearson,
            'roc_auc': roc_auc,
            'pr_auc': pr_auc,
            'best_threshold': best_threshold,
            'best_f1': best_f1,
            'precision': precision,
            'recall': recall,
            'cosine_scores': cosine_scores,
            'labels': labels_array
        }
        
        self.results[model_name] = results
        
        # Print summary
        if verbose:
            print("\nMetrics:")
            print(f"  Spearman Correlation: {eval_spearman:.4f}")
            print(f"  Pearson Correlation:  {eval_pearson:.4f}")
            if roc_auc is not None:
                print(f"  ROC-AUC:             {roc_auc:.4f}")
                print(f"  PR-AUC:              {pr_auc:.4f}")
            print(f"\nClassification (threshold={best_threshold:.3f}):")
            print(f"  F1 Score:            {best_f1:.4f}")
            print(f"  Precision:           {precision:.4f}")
            print(f"  Recall:              {recall:.4f}")
        
        return results
    
    def compare_models(self) -> pd.DataFrame:
        """
        Create a comparison table of all evaluated models.
        """
        comparison_data = []
        
        for model_name, results in self.results.items():
            comparison_data.append({
                'Model': model_name,
                'Spearman': results['spearman'],
                'Pearson': results['pearson'],
                'ROC-AUC': results['roc_auc'],
                'PR-AUC': results['pr_auc'],
                'Best F1': results['best_f1'],
                'Threshold': results['best_threshold'],
                'Precision': results['precision'],
                'Recall': results['recall']
            })
        
        return pd.DataFrame(comparison_data)
    
    def plot_roc_curves(self, figsize=(10, 8)):
        """
        Plot ROC curves for all evaluated models.
        """
        plt.figure(figsize=figsize)
        
        for model_name, results in self.results.items():
            if results['roc_auc'] is not None:
                fpr, tpr, _ = roc_curve(results['labels'], results['cosine_scores'])
                plt.plot(fpr, tpr, label=f"{model_name} (AUC={results['roc_auc']:.4f})", linewidth=2)
        
        plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC=0.5000)', linewidth=1)
        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
        plt.legend(loc='lower right', fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def plot_pr_curves(self, figsize=(10, 8)):
        """
        Plot Precision-Recall curves for all evaluated models.
        """
        plt.figure(figsize=figsize)
        
        for model_name, results in self.results.items():
            if results['pr_auc'] is not None:
                precision, recall, _ = precision_recall_curve(results['labels'], results['cosine_scores'])
                plt.plot(recall, precision, label=f"{model_name} (AP={results['pr_auc']:.4f})", linewidth=2)
        
        # Baseline (proportion of positive class)
        baseline = np.mean(list(self.results.values())[0]['labels'])
        plt.plot([0, 1], [baseline, baseline], 'k--', label=f'Random (AP={baseline:.4f})', linewidth=1)
        
        plt.xlabel('Recall', fontsize=12)
        plt.ylabel('Precision', fontsize=12)
        plt.title('Precision-Recall Curves - Model Comparison', fontsize=14, fontweight='bold')
        plt.legend(loc='lower left', fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    def plot_score_distributions(self, figsize=(14, 6)):
        """
        Plot similarity score distributions for all evaluated models.
        """
        n_models = len(self.results)
        fig, axes = plt.subplots(1, n_models, figsize=figsize, sharey=True)
        
        if n_models == 1:
            axes = [axes]
        
        for idx, (model_name, results) in enumerate(self.results.items()):
            ax = axes[idx]
            
            # Separate scores by label
            pos_scores = results['cosine_scores'][results['labels'] == 1]
            neg_scores = results['cosine_scores'][results['labels'] == 0]
            
            # Plot histograms
            ax.hist(neg_scores, bins=50, alpha=0.6, label='Negative (0)', color='red', edgecolor='black')
            ax.hist(pos_scores, bins=50, alpha=0.6, label='Positive (1)', color='green', edgecolor='black')
            
            # Mark optimal threshold
            ax.axvline(results['best_threshold'], color='blue', linestyle='--', linewidth=2,
                      label=f"Threshold={results['best_threshold']:.3f}")
            
            ax.set_xlabel('Cosine Similarity Score', fontsize=11)
            if idx == 0:
                ax.set_ylabel('Frequency', fontsize=11)
            ax.set_title(model_name, fontsize=12, fontweight='bold')
            ax.legend(fontsize=9)
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def save_results(self, output_path: Path):
        """
        Save evaluation results to CSV.
        """
        comparison_df = self.compare_models()
        comparison_df.to_csv(output_path, index=False)
        print(f"\n✓ Results saved to: {output_path}")

print("✓ ModelEvaluator class defined")

## 6. Evaluate Baseline Model (Raw MPNet)

In [ ]:
# Initialize evaluator
evaluator = ModelEvaluator(batch_size=BATCH_SIZE)

# Load baseline model (raw MPNet without fine-tuning)
print("Loading baseline model: sentence-transformers/all-mpnet-base-v2")
baseline_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

# Evaluate baseline
baseline_results = evaluator.evaluate_model(
    model=baseline_model,
    model_name='Baseline (Raw MPNet)',
    texts1=test_texts1,
    texts2=test_texts2,
    labels=test_labels,
    verbose=True
)

print("\n✓ Baseline evaluation complete")

## 7. Evaluate Fine-Tuned Models

Select models to evaluate from the list above.

In [ ]:
# Select models to evaluate (modify this list as needed)
# Examples:
#   - 'v6_refactored_finetuned'
#   - 'real_servicenow_finetuned_mpnet'
#   - 'real_servicenow_finetuned_nomic'
#   - 'all-mpnet-finetuned-v6'

models_to_evaluate = [
    'v6_refactored_finetuned',
    'real_servicenow_finetuned_mpnet',
    'real_servicenow_finetuned_nomic',
    'all-mpnet-finetuned-v6'
]

print(f"Selected {len(models_to_evaluate)} models for evaluation:\n")
for model_name in models_to_evaluate:
    if model_name in available_models:
        print(f"  ✓ {model_name}")
    else:
        print(f"  ✗ {model_name} (not found)")

In [ ]:
# Evaluate each selected model
for model_name in models_to_evaluate:
    if model_name not in available_models:
        print(f"\n⚠ Skipping {model_name} - not found in models directory")
        continue
    
    model_path = available_models[model_name]
    
    try:
        # Load model
        print(f"\nLoading model from: {model_path}")
        model = SentenceTransformer(str(model_path), device=device)
        
        # Evaluate
        evaluator.evaluate_model(
            model=model,
            model_name=model_name,
            texts1=test_texts1,
            texts2=test_texts2,
            labels=test_labels,
            verbose=True
        )
        
        # Clean up
        del model
        if device == 'cuda':
            torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"\n✗ Error evaluating {model_name}: {e}")
        continue

print("\n" + "=" * 60)
print("✓ All model evaluations complete")
print("=" * 60)

## 8. Model Comparison and Visualization

In [ ]:
# Generate comparison table
comparison_df = evaluator.compare_models()

# Sort by Spearman correlation (primary metric)
comparison_df = comparison_df.sort_values('Spearman', ascending=False)

print("\n" + "=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)
print(comparison_df.to_string(index=False))
print("=" * 80)

In [ ]:
# Styled DataFrame for better visualization
styled_df = comparison_df.style.background_gradient(
    subset=['Spearman', 'Pearson', 'ROC-AUC', 'PR-AUC', 'Best F1'],
    cmap='RdYlGn',
    vmin=0.0,
    vmax=1.0
).format({
    'Spearman': '{:.4f}',
    'Pearson': '{:.4f}',
    'ROC-AUC': '{:.4f}',
    'PR-AUC': '{:.4f}',
    'Best F1': '{:.4f}',
    'Threshold': '{:.3f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}'
})

styled_df

In [ ]:
# Plot ROC curves
evaluator.plot_roc_curves(figsize=(10, 8))

In [ ]:
# Plot Precision-Recall curves
evaluator.plot_pr_curves(figsize=(10, 8))

In [ ]:
# Plot similarity score distributions
evaluator.plot_score_distributions(figsize=(16, 6))

## 9. Metric Analysis by Category

In [ ]:
# Bar chart comparing key metrics across models
metrics_to_plot = ['Spearman', 'Pearson', 'ROC-AUC', 'PR-AUC', 'Best F1']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    # Sort by metric value
    sorted_df = comparison_df.sort_values(metric, ascending=True)
    
    # Create horizontal bar chart
    colors = ['red' if model == 'Baseline (Raw MPNet)' else 'steelblue' 
              for model in sorted_df['Model']]
    
    ax.barh(sorted_df['Model'], sorted_df[metric], color=colors, edgecolor='black')
    ax.set_xlabel(metric, fontsize=12, fontweight='bold')
    ax.set_xlim(0, 1)
    ax.grid(True, alpha=0.3, axis='x')
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    
    # Add value labels
    for i, (model, value) in enumerate(zip(sorted_df['Model'], sorted_df[metric])):
        if pd.notna(value):
            ax.text(value + 0.01, i, f'{value:.4f}', va='center', fontsize=9)

# Remove extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

## 10. Performance Delta vs Baseline

In [ ]:
# Calculate improvement over baseline
baseline_name = 'Baseline (Raw MPNet)'

if baseline_name in evaluator.results:
    baseline_metrics = {
        'Spearman': evaluator.results[baseline_name]['spearman'],
        'Pearson': evaluator.results[baseline_name]['pearson'],
        'ROC-AUC': evaluator.results[baseline_name]['roc_auc'],
        'PR-AUC': evaluator.results[baseline_name]['pr_auc'],
        'Best F1': evaluator.results[baseline_name]['best_f1']
    }
    
    improvement_data = []
    
    for model_name, results in evaluator.results.items():
        if model_name == baseline_name:
            continue
        
        improvement_data.append({
            'Model': model_name,
            'Δ Spearman': results['spearman'] - baseline_metrics['Spearman'],
            'Δ Pearson': results['pearson'] - baseline_metrics['Pearson'],
            'Δ ROC-AUC': results['roc_auc'] - baseline_metrics['ROC-AUC'] if results['roc_auc'] else None,
            'Δ PR-AUC': results['pr_auc'] - baseline_metrics['PR-AUC'] if results['pr_auc'] else None,
            'Δ F1': results['best_f1'] - baseline_metrics['Best F1'],
        })
    
    improvement_df = pd.DataFrame(improvement_data)
    improvement_df = improvement_df.sort_values('Δ Spearman', ascending=False)
    
    print("\n" + "=" * 80)
    print("IMPROVEMENT OVER BASELINE (Raw MPNet)")
    print("=" * 80)
    print(improvement_df.to_string(index=False))
    print("=" * 80)
    
    # Styled version
    styled_improvement = improvement_df.style.background_gradient(
        subset=['Δ Spearman', 'Δ Pearson', 'Δ ROC-AUC', 'Δ PR-AUC', 'Δ F1'],
        cmap='RdYlGn',
        vmin=-0.1,
        vmax=0.3
    ).format({
        'Δ Spearman': '{:+.4f}',
        'Δ Pearson': '{:+.4f}',
        'Δ ROC-AUC': '{:+.4f}',
        'Δ PR-AUC': '{:+.4f}',
        'Δ F1': '{:+.4f}'
    })
    
    display(styled_improvement)
else:
    print("Baseline model not found in results")

## 11. Export Results

In [ ]:
# Save comparison results
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_file = RESULTS_DIR / f'model_evaluation_{timestamp}.csv'

evaluator.save_results(results_file)

# Also save improvement metrics if available
if 'improvement_df' in locals():
    improvement_file = RESULTS_DIR / f'model_improvement_{timestamp}.csv'
    improvement_df.to_csv(improvement_file, index=False)
    print(f"✓ Improvement metrics saved to: {improvement_file}")

## 12. Summary and Recommendations

In [ ]:
# Generate summary report
print("\n" + "=" * 80)
print("EVALUATION SUMMARY")
print("=" * 80)

print(f"\nTest Set Size: {len(test_labels)} pairs")
print(f"  - Positive pairs: {sum(test_labels)} ({100*sum(test_labels)/len(test_labels):.1f}%)")
print(f"  - Negative pairs: {len(test_labels) - sum(test_labels)} ({100*(len(test_labels)-sum(test_labels))/len(test_labels):.1f}%)")

print(f"\nModels Evaluated: {len(evaluator.results)}")
for model_name in evaluator.results.keys():
    print(f"  - {model_name}")

# Best model for each metric
print("\n" + "-" * 80)
print("BEST PERFORMING MODELS BY METRIC")
print("-" * 80)

for metric in ['Spearman', 'Pearson', 'ROC-AUC', 'PR-AUC', 'Best F1']:
    best_row = comparison_df.loc[comparison_df[metric].idxmax()]
    print(f"\n{metric:15s}: {best_row['Model']}")
    print(f"                 Value: {best_row[metric]:.4f}")

# Overall recommendation
print("\n" + "-" * 80)
print("RECOMMENDATION")
print("-" * 80)

# Use Spearman as primary metric (most important for ranking)
best_overall = comparison_df.loc[comparison_df['Spearman'].idxmax()]
print(f"\nBest Overall Model: {best_overall['Model']}")
print(f"  Spearman: {best_overall['Spearman']:.4f}")
print(f"  Pearson:  {best_overall['Pearson']:.4f}")
print(f"  ROC-AUC:  {best_overall['ROC-AUC']:.4f}")
print(f"  PR-AUC:   {best_overall['PR-AUC']:.4f}")

if baseline_name in evaluator.results:
    baseline_spearman = evaluator.results[baseline_name]['spearman']
    improvement_pct = 100 * (best_overall['Spearman'] - baseline_spearman) / baseline_spearman
    print(f"\nImprovement over baseline: {improvement_pct:+.1f}%")

print("\n" + "=" * 80)

## 13. Optional: Evaluate Additional Models

You can add more models to evaluate by modifying the `models_to_evaluate` list in cell 7 and re-running from there.

In [ ]:
# This cell can be used to quickly evaluate a single additional model
# Uncomment and modify as needed:

# additional_model_name = 'all-mpnet-finetuned-v5'
# 
# if additional_model_name in available_models:
#     model_path = available_models[additional_model_name]
#     model = SentenceTransformer(str(model_path), device=device)
#     
#     evaluator.evaluate_model(
#         model=model,
#         model_name=additional_model_name,
#         texts1=test_texts1,
#         texts2=test_texts2,
#         labels=test_labels,
#         verbose=True
#     )
#     
#     del model
#     if device == 'cuda':
#         torch.cuda.empty_cache()
# else:
#     print(f"Model '{additional_model_name}' not found")